# Factor Model

This demo is written under Python 3.4 environment using Jupyter Notebook through [Anaconda](https://anaconda.org/).

<font color='red'>This demo is for course use only. Do not distribute.</font>

## 1. Introduction to WRDS and Download Data

## 2. Formulas

$Market Equity(ME) = Price * Shares$
<br>

$Book Equity(BE) = Stockholders' equity + Deferred taxes and tax credit - Preferred stock$
<br>

$Profitability = Sales/Turnover - Cost of goods sold - Selling, general and admin expense - Interest and related expense$
<br>

For more details of factor definition, please ta ke a look at [Ken French Library](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/variable_definitions.html).

# 3. Data Preprocessing

This part is using Python 3.4 and Jupyter Notebook for demonstration only. You can choose other programming languages as you like.

When doing data analysis in Python, [Numpy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) and [Pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html) will be very helpful.

In [1]:
# import some useful packages
# these packages will be installed by default if you are using anaconda
import pandas as pd
import numpy as np
import datetime

## 3.1 Read Downloaded Data

In [2]:
# current working directory: UTA/Code
# save downloaded data under: UTA/Code/Data/
raw_df = pd.read_csv("raw_data.csv")

In [3]:
# look at column types
print(raw_df.dtypes)
print(raw_df.describe())
raw_df.head()

gvkey         int64
datadate     object
fyearq        int64
fqtr          int64
indfmt       object
consol       object
popsrc       object
datafmt      object
tic          object
conm         object
curcdq       object
datacqtr     object
datafqtr     object
cogsq       float64
cshoq       float64
pstkq       float64
saleq       float64
teqq        float64
txditcq     float64
xintq       float64
xsgaq       float64
exchg       float64
costat       object
prccq       float64
dtype: object
               gvkey         fyearq           fqtr          cogsq  \
count  160818.000000  160818.000000  160818.000000  117883.000000   
mean    88752.423062    2015.043366       2.485742     734.558955   
std     76867.521251       1.415940       1.112868    3688.247064   
min      1004.000000    2012.000000       1.000000   -4788.000000   
25%     19143.000000    2014.000000       1.000000       2.225000   
50%     61796.000000    2015.000000       2.000000      29.215000   
75%    170912.000000   

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,conm,...,cshoq,pstkq,saleq,teqq,txditcq,xintq,xsgaq,exchg,costat,prccq
0,1004,05/31/2013,2012,4,INDL,C,D,STD,AIR,AAR CORP,...,39.382,0.0,583.6,919.5,138.2,10.0,52.5,11.0,A,20.06
1,1004,08/31/2013,2013,1,INDL,C,D,STD,AIR,AAR CORP,...,39.582,0.0,514.5,939.4,146.8,11.0,47.7,11.0,A,25.09
2,1004,11/30/2013,2013,2,INDL,C,D,STD,AIR,AAR CORP,...,39.600,0.0,374.2,967.2,153.5,3.8,34.9,11.0,A,31.22
3,1004,02/28/2014,2013,3,INDL,C,D,STD,AIR,AAR CORP,...,39.569,0.0,399.8,988.1,143.2,7.1,37.7,11.0,A,28.90
4,1004,05/31/2014,2013,4,INDL,C,D,STD,AIR,AAR CORP,...,39.560,0.0,420.6,1000.7,162.2,6.4,46.0,11.0,A,24.30


## 3.2 Remove unnecessary columns and rename columns

In [4]:
# column selection
raw_df = raw_df.loc[:, ["datadate", "tic", "conm", "cogsq", "cshoq", "pstkq", "saleq", "teqq", "txditcq", "xintq", "xsgaq", 
                        "exchg", "prccq"]].copy()

# rename column
# it's NOT a good habit to have white space inside column names in Python
# I copy paste column names from R to Python, so I did it anyway...
raw_df.columns = ["Date", "Ticker", "Company", "Cost of Goods Sold", "Common Shares Outstanding", "Preferred Stock Capital", 
                  "Sales/Turnover (Net)", "Stockholders Equity", "Deferred Taxes and Investment Tax Credit", 
                  "Interest and Related Expense", "Selling, General and Administrative Expenses", 
                  "Stock Exchange Code", "Close Price"]

raw_df.head()

,Date,Ticker,Company,Cost of Goods Sold,Common Shares Outstanding,Preferred Stock Capital,Sales/Turnover (Net),Stockholders Equity,Deferred Taxes and Investment Tax Credit,Interest and Related Expense,"Selling, General and Administrative Expenses",Stock Exchange Code,Close Price
0,05/31/2013,AIR,AAR CORP,465.9,39.382,0.0,583.6,919.5,138.2,10.0,52.5,11.0,20.06
1,08/31/2013,AIR,AAR CORP,401.8,39.582,0.0,514.5,939.4,146.8,11.0,47.7,11.0,25.09
2,11/30/2013,AIR,AAR CORP,281.0,39.600,0.0,374.2,967.2,153.5,3.8,34.9,11.0,31.22
3,02/28/2014,AIR,AAR CORP,305.0,39.569,0.0,399.8,988.1,143.2,7.1,37.7,11.0,28.90
4,05/31/2014,AIR,AAR CORP,319.0,39.560,0.0,420.6,1000.7,162.2,6.4,46.0,11.0,24.30


## 3.3 Handle missing data

In data analysis, you will meet with missing data inevitably. How to handle missing data is an art.
<br>

We could replace missing data using [fillna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html).
<br>

We could also simply drop row contains missing data by using [dropna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html).

Make realistic assumptions and defend your assumptions!

In [5]:
# how to use help inside Jupyter Notebook
help(pd.DataFrame.dropna)

Help on function dropna in module pandas.core.frame:

dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
    Return object with labels on given axis omitted where alternately any
    or all of the data are missing
    
    Parameters
    ----------
    axis : {0 or 'index', 1 or 'columns'}, or tuple/list thereof
        Pass tuple or list to drop on multiple axes
    how : {'any', 'all'}
        * any : if any NA values are present, drop that label
        * all : if all values are NA, drop that label
    thresh : int, default None
        int value : require that many non-NA values
    subset : array-like
        Labels along other axis to consider, e.g. if you are dropping rows
        these would be a list of columns to include
    inplace : boolean, default False
        If True, do operation inplace and return None.
    
    Returns
    -------
    dropped : DataFrame
    
    Examples
    --------
    >>> df = pd.DataFrame([[np.nan, 2, np.nan, 0], [3, 4, np.

In [6]:
# drop if price or shares is missing
raw_df.dropna(axis=0, how='any', subset=['Close Price', 'Common Shares Outstanding'], inplace=True)

## 3.4 Calculate ME, BE and Profitability

In [7]:
# market equity
raw_df['ME'] = raw_df['Close Price'] * raw_df['Common Shares Outstanding']
# book equity
raw_df['BE'] = raw_df['Stockholders Equity'] + raw_df['Deferred Taxes and Investment Tax Credit'].fillna(0) -\
raw_df['Preferred Stock Capital'].fillna(0)
# book to market ratio
raw_df['BE/ME'] = raw_df['BE'] / raw_df['ME']
# profitability
raw_df['PB'] = raw_df['Sales/Turnover (Net)'] - raw_df[['Cost of Goods Sold', 'Selling, General and Administrative Expenses',\
                                                        'Interest and Related Expense']].sum(axis=1, skipna=True)

# get cleaned dataset
clean_df = raw_df.iloc[:, [0,1,2,11,13,14,15,16]].copy()

**Note: the last step to make a copy is very important in Python in order to prevent [SettingwithCopyWarning](https://www.dataquest.io/blog/settingwithcopywarning/)**

In [8]:
# DO NOT use this if your dataset is huge
clean_df

,Date,Ticker,Company,Stock Exchange Code,ME,BE,BE/ME,PB
0,05/31/2013,AIR,AAR CORP,11.0,790.002920,1057.700,1.338856,55.200
1,08/31/2013,AIR,AAR CORP,11.0,993.112380,1086.200,1.093733,54.000
2,11/30/2013,AIR,AAR CORP,11.0,1236.312000,1120.700,0.906486,54.500
3,02/28/2014,AIR,AAR CORP,11.0,1143.544100,1131.300,0.989293,50.000
4,05/31/2014,AIR,AAR CORP,11.0,961.308000,1162.900,1.209706,49.200
5,08/31/2014,AIR,AAR CORP,11.0,1104.620600,1167.500,1.056924,36.900
6,11/30/2014,AIR,AAR CORP,11.0,1019.843330,1171.100,1.148314,38.600
7,02/28/2015,AIR,AAR CORP,11.0,1170.590400,1080.900,0.923380,20.600
8,05/31/2015,AIR,AAR CORP,11.0,1046.395420,949.700,0.907592,-52.000
9,08/31/2015,AIR,AAR CORP,11.0,856.561980,958.100,1.118541,29.800


As we can see, there are some NaNs in ME or BE or PB columns. Therefore, we are going to drop entire rows in this case.

In [9]:
clean_df.dropna(axis=0, how='any', subset=['ME', 'BE', 'PB'], inplace=True)

In [10]:
clean_df

,Date,Ticker,Company,Stock Exchange Code,ME,BE,BE/ME,PB
0,05/31/2013,AIR,AAR CORP,11.0,790.002920,1057.700,1.338856,55.200
1,08/31/2013,AIR,AAR CORP,11.0,993.112380,1086.200,1.093733,54.000
2,11/30/2013,AIR,AAR CORP,11.0,1236.312000,1120.700,0.906486,54.500
3,02/28/2014,AIR,AAR CORP,11.0,1143.544100,1131.300,0.989293,50.000
4,05/31/2014,AIR,AAR CORP,11.0,961.308000,1162.900,1.209706,49.200
5,08/31/2014,AIR,AAR CORP,11.0,1104.620600,1167.500,1.056924,36.900
6,11/30/2014,AIR,AAR CORP,11.0,1019.843330,1171.100,1.148314,38.600
7,02/28/2015,AIR,AAR CORP,11.0,1170.590400,1080.900,0.923380,20.600
8,05/31/2015,AIR,AAR CORP,11.0,1046.395420,949.700,0.907592,-52.000
9,08/31/2015,AIR,AAR CORP,11.0,856.561980,958.100,1.118541,29.800


# 4. Calculate Factor Break Points

Below is one example of portfolio construction. You can modify the code to make your own choices.
<br>

Rebalance Frequency: Annual Rebalance

Rebalance Date: end of March
<br>

Details: 

In **May** or **Jun** of year $t-1$, all NYSE stocks are ranked on size. The median NYSE size is then used to split NYSE, Amex and NASDAQ stocks into two groups, small and big (S and B).

Also rank NYSE, Amex and NASDAQ stocks into two book-to-market equity groups based on the ranked values of BE/ME for NYSE stocks in **Nov** or **Dec** of year $t-1$. If BE/ME ratio is high, then stock is considered value stock (V). If BE/ME is low, the stock is considered growth stock (G).

The methdology above is just one example. For more details, please look at [Ken French Library](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_beme_breakpoints.html)

In [11]:
# convert Date column to datetime
clean_df.loc[:, 'Date'] = pd.to_datetime(clean_df.loc[:, 'Date'], format='%m/%d/%Y')

In [12]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104555 entries, 0 to 160817
Data columns (total 8 columns):
Date                   104555 non-null datetime64[ns]
Ticker                 104555 non-null object
Company                104555 non-null object
Stock Exchange Code    104555 non-null float64
ME                     104555 non-null float64
BE                     104555 non-null float64
BE/ME                  104555 non-null float64
PB                     104555 non-null float64
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 7.2+ MB


In [13]:
# get data with month equals to May or Jun and listed on NYSE exchange for ME breakpoint calculation
me_df = clean_df.loc[((clean_df['Date'].dt.month == 5) | (clean_df['Date'].dt.month == 6))\
                     & (clean_df['Stock Exchange Code'] == 11)].copy()

In [14]:
me_df

,Date,Ticker,Company,Stock Exchange Code,ME,BE,BE/ME,PB
0,2013-05-31,AIR,AAR CORP,11.0,790.00292,1057.700,1.338856,55.200
4,2014-05-31,AIR,AAR CORP,11.0,961.30800,1162.900,1.209706,49.200
8,2015-05-31,AIR,AAR CORP,11.0,1046.39542,949.700,0.907592,-52.000
12,2016-05-31,AIR,AAR CORP,11.0,842.51115,900.100,1.068354,35.200
16,2017-05-31,AIR,AAR CORP,11.0,1200.32876,951.400,0.792616,37.600
60,2013-05-31,ASA,ASA GOLD AND PRECIOUS METALS,11.0,276.61860,302.358,1.093050,1.047
64,2014-05-31,ASA,ASA GOLD AND PRECIOUS METALS,11.0,251.54160,266.511,1.059511,-0.094
68,2015-05-31,ASA,ASA GOLD AND PRECIOUS METALS,11.0,205.82430,227.606,1.105827,-0.150
72,2016-05-31,ASA,ASA GOLD AND PRECIOUS METALS,11.0,227.04330,260.266,1.146328,-0.328
76,2017-05-31,ASA,ASA GOLD AND PRECIOUS METALS,11.0,229.16520,260.144,1.135181,-0.071


In [15]:
for yy in list(sorted(set(me_df['Date'].dt.year))):
    tmp_df = me_df.loc[me_df['Date'].dt.year == yy].copy()
    
    me_br_pt = np.nanmedian(tmp_df['ME'])
    
    clean_df.loc[clean_df['Date'].dt.year == yy, 'Group1'] = np.where(clean_df.loc[clean_df['Date'].dt.year == yy, 'ME'] <= me_br_pt, "Small", "Big")

In [16]:
# get data with month equals to Nov or Dec and listed on NYSE exchange for BE/ME breakpoint calculation
be_df = clean_df.loc[((clean_df['Date'].dt.month == 11) | (clean_df['Date'].dt.month == 12))\
                     & (clean_df['Stock Exchange Code'] == 11)].copy()

In [17]:
for yy in list(sorted(set(be_df['Date'].dt.year))):
    tmp_df = be_df.loc[be_df['Date'].dt.year == yy].copy()
    
    be_br_pt = np.nanmedian(tmp_df['BE/ME'])
    
    clean_df.loc[clean_df['Date'].dt.year == yy, 'Group2'] = np.where(clean_df.loc[clean_df['Date'].dt.year == yy, 'BE/ME'] <= be_br_pt, "Growth", "Value")

We are going to drop NaNs in Group1 or Group2. **Q: Why do we have NaNs in Group1 or Group2 when info shows they are non-null?**

In [18]:
category = clean_df.loc[(clean_df['Group1'] != 'nan') | (clean_df['Group2'] != 'nan')].copy().reset_index(drop=True)

In [19]:
category

,Date,Ticker,Company,Stock Exchange Code,ME,BE,BE/ME,PB,Group1,Group2
0,2013-05-31,AIR,AAR CORP,11.0,790.002920,1057.700,1.338856,55.200,Small,Value
1,2013-08-31,AIR,AAR CORP,11.0,993.112380,1086.200,1.093733,54.000,Small,Value
2,2013-11-30,AIR,AAR CORP,11.0,1236.312000,1120.700,0.906486,54.500,Small,Value
3,2014-02-28,AIR,AAR CORP,11.0,1143.544100,1131.300,0.989293,50.000,Small,Value
4,2014-05-31,AIR,AAR CORP,11.0,961.308000,1162.900,1.209706,49.200,Small,Value
5,2014-08-31,AIR,AAR CORP,11.0,1104.620600,1167.500,1.056924,36.900,Small,Value
6,2014-11-30,AIR,AAR CORP,11.0,1019.843330,1171.100,1.148314,38.600,Small,Value
7,2015-02-28,AIR,AAR CORP,11.0,1170.590400,1080.900,0.923380,20.600,Small,Value
8,2015-05-31,AIR,AAR CORP,11.0,1046.395420,949.700,0.907592,-52.000,Small,Value
9,2015-08-31,AIR,AAR CORP,11.0,856.561980,958.100,1.118541,29.800,Small,Value


# 5. Simulation

The simulation begins in 2014 March and ends in 2017 December. The stocks within each category will be equally weighted. You can choose other weighting methods.

## 5.1 Download Return Data

# 5.2 Return Data Preprocessing

You can difinitely have different ways of implementing logic here. The code below is just a reference.

In [20]:
return_df = pd.read_csv("monthly_total_return.csv")

return_df = return_df.loc[:, ["date", "TICKER", "COMNAM", "sprtrn", "ewretd", "vwretd"]].copy()

# rename column
# The column names below are copied from R to Python, however it is recommended to rename columns without white space in python.
return_df.columns = ["Date", "Ticker", "Company", "S&P_RTN", 'Equally_wt_RTN', "Volume_wt_RTN"]

# drop row that contain NA
return_df.dropna(inplace=True)

return_df.head()

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN
0,03/28/2013,EGAS,GAS NATURAL INC,0.035988,0.031419,0.035274
1,04/30/2013,EGAS,GAS NATURAL INC,0.018086,0.002477,0.014945
2,05/31/2013,EGAS,GAS NATURAL INC,0.020763,0.023985,0.019067
3,06/28/2013,EGAS,GAS NATURAL INC,-0.014999,-0.013012,-0.015039
4,07/31/2013,EGAS,GAS NATURAL INC,0.049462,0.054898,0.052681


In [21]:
# change Date column to datetime
return_df.loc[:, 'Date'] = pd.to_datetime(return_df.loc[:, 'Date'], format='%m/%d/%Y')

# sort by Ticker and Date column
return_df.sort_values(['Ticker', 'Date'], ascending=[True, True], inplace=True)

In [22]:
return_df.head()

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN
273418,2013-03-28,A,AGILENT TECHNOLOGIES INC,0.035988,0.031419,0.035274
273419,2013-04-30,A,AGILENT TECHNOLOGIES INC,0.018086,0.002477,0.014945
273420,2013-05-31,A,AGILENT TECHNOLOGIES INC,0.020763,0.023985,0.019067
273421,2013-06-28,A,AGILENT TECHNOLOGIES INC,-0.014999,-0.013012,-0.015039
273422,2013-07-31,A,AGILENT TECHNOLOGIES INC,0.049462,0.054898,0.052681


Here I use S_Period to indicate 3 simulation periods and compound the stocks in the same simulation period to get annualized returns.

In [23]:
# set a default value
return_df['S_Period'] = 0
# set values based on multiple conditions
return_df.loc[(return_df['Date']>='2014-04-30') & (return_df['Date']<='2015-03-31'), 'S_Period'] = 1
return_df.loc[(return_df['Date']>='2015-04-30') & (return_df['Date']<='2016-03-31'), 'S_Period'] = 2
return_df.loc[(return_df['Date']>='2016-04-30') & (return_df['Date']<='2017-03-31'), 'S_Period'] = 3
return_df.loc[(return_df['Date']>='2017-04-30') & (return_df['Date']<='2018-03-31'), 'S_Period'] = 4

In [24]:
# check
set(return_df.loc[return_df['S_Period']==0, 'Date'])

{Timestamp('2013-03-28 00:00:00'),
 Timestamp('2013-04-30 00:00:00'),
 Timestamp('2013-05-31 00:00:00'),
 Timestamp('2013-06-28 00:00:00'),
 Timestamp('2013-07-31 00:00:00'),
 Timestamp('2013-08-30 00:00:00'),
 Timestamp('2013-09-30 00:00:00'),
 Timestamp('2013-10-31 00:00:00'),
 Timestamp('2013-11-29 00:00:00'),
 Timestamp('2013-12-31 00:00:00'),
 Timestamp('2014-01-31 00:00:00'),
 Timestamp('2014-02-28 00:00:00'),
 Timestamp('2014-03-31 00:00:00'),
 Timestamp('2016-04-29 00:00:00'),
 Timestamp('2017-04-28 00:00:00')}

**Note: since we drop NA in *M Total Return* column, for some stocks we may not have complete 12 month returns**.
<br>

In that case, I simply drop those stocks for those simulation periods.

In [25]:
filtered_return = return_df.groupby(['Ticker', 'S_Period']).filter(lambda x: len(x) == 12).copy()

In [26]:
filtered_return.reset_index(drop=True, inplace=True)
filtered_return.tail()

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN,S_Period
157171,2015-11-30,ZX,CHINA ZENIX AUTO INTL LTD,0.000505,0.007333,0.002442,2
157172,2015-12-31,ZX,CHINA ZENIX AUTO INTL LTD,-0.017530,-0.035887,-0.022257,2
157173,2016-01-29,ZX,CHINA ZENIX AUTO INTL LTD,-0.050735,-0.074951,-0.057035,2
157174,2016-02-29,ZX,CHINA ZENIX AUTO INTL LTD,-0.004128,0.005557,0.000682,2
157175,2016-03-31,ZX,CHINA ZENIX AUTO INTL LTD,0.065991,0.078191,0.070460,2


## 5.3 Aggragate Monthly Return

In [28]:
filtered_return.loc[:, 'Equally_wt_RTN'] = filtered_return.loc[:, 'Equally_wt_RTN']/100+1

In [30]:
filtered_return['Annual_Return'] = filtered_return.groupby(['Ticker', 'S_Period'])['Equally_wt_RTN'].transform('prod')

In [31]:
filtered_return.head(15)

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN,S_Period,Annual_Return
0,2014-04-30,A,AGILENT TECHNOLOGIES INC,0.006201,0.999777,0.001661,1,1.000188
1,2014-05-30,A,AGILENT TECHNOLOGIES INC,0.021030,1.000066,0.020219,1,1.000188
2,2014-06-30,A,AGILENT TECHNOLOGIES INC,0.019058,1.000395,0.027941,1,1.000188
3,2014-07-31,A,AGILENT TECHNOLOGIES INC,-0.015080,0.999648,-0.020523,1,1.000188
4,2014-08-29,A,AGILENT TECHNOLOGIES INC,0.037655,1.000337,0.040186,1,1.000188
5,2014-09-30,A,AGILENT TECHNOLOGIES INC,-0.015514,0.999550,-0.025129,1,1.000188
6,2014-10-31,A,AGILENT TECHNOLOGIES INC,0.023201,1.000142,0.021184,1,1.000188
7,2014-11-28,A,AGILENT TECHNOLOGIES INC,0.024534,0.999975,0.021150,1,1.000188
8,2014-12-31,A,AGILENT TECHNOLOGIES INC,-0.004189,0.999998,-0.003616,1,1.000188
9,2015-01-30,A,AGILENT TECHNOLOGIES INC,-0.031041,0.999809,-0.027152,1,1.000188


**Note: All we need is rebalancing date (YYYY-03-31) information. So let's drop some rows.**

In [32]:
annual_return = filtered_return.loc[filtered_return['Date'].dt.month==3].reset_index(drop=True).copy()

In [33]:
annual_return.head()

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN,S_Period,Annual_Return
0,2015-03-31,A,AGILENT TECHNOLOGIES INC,-0.017396,0.999949,-0.010439,1,1.000188
1,2016-03-31,A,AGILENT TECHNOLOGIES INC,0.065991,1.000782,0.070460,2,0.999143
2,2015-03-31,AA,ALCOA INC,-0.017396,0.999949,-0.010439,1,1.000188
3,2016-03-31,AA,ALCOA INC,0.065991,1.000782,0.070460,2,0.999143
4,2016-03-31,AAC,A A C HOLDINGS INC,0.065991,1.000782,0.070460,2,0.999143


## 5.4 Merge Category Table and Return Table

In [42]:
final_res = pd.merge(annual_return, category, on=['Date', 'Ticker', 'Company'], how='inner')
final_res

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN,S_Period,Annual_Return,Stock Exchange Code,ME,BE,BE/ME,PB,Group1,Group2
0,2015-03-31,AAL,AMERICAN AIRLINES GROUP INC,-0.017396,0.999949,-0.010439,1,1.000188,14.0,36618.81678,2764.000,0.075480,1697.000,Big,Growth
1,2016-03-31,AAL,AMERICAN AIRLINES GROUP INC,0.065991,1.000782,0.070460,2,0.999143,14.0,24017.83458,4710.000,0.196104,1612.000,Big,Growth
2,2015-03-31,AAME,ATLANTIC AMERICAN CORP,-0.017396,0.999949,-0.010439,1,1.000188,14.0,82.93740,110.774,1.335634,1.293,Small,Value
3,2016-03-31,AAME,ATLANTIC AMERICAN CORP,0.065991,1.000782,0.070460,2,0.999143,14.0,90.73995,103.944,1.145515,1.667,Small,Value
4,2015-03-31,AAOI,APPLIED OPTOELECTRONICS INC,-0.017396,0.999949,-0.010439,1,1.000188,14.0,206.71484,114.907,0.555872,0.975,Small,Growth
5,2016-03-31,AAOI,APPLIED OPTOELECTRONICS INC,0.065991,1.000782,0.070460,2,0.999143,14.0,254.51370,165.879,0.651749,1.173,Small,Value
6,2015-03-31,AAON,AAON INC,-0.017396,0.999949,-0.010439,1,1.000188,14.0,1328.81463,195.484,0.147112,16.300,Small,Growth
7,2016-03-31,AAON,AAON INC,0.065991,1.000782,0.070460,2,0.999143,14.0,1485.68000,196.346,0.132159,19.989,Small,Growth
8,2015-03-31,AAP,ADVANCE AUTO PARTS INC,-0.017396,0.999949,-0.010439,1,1.000188,11.0,10952.51792,2146.588,0.195990,356.703,Big,Growth
9,2016-03-31,AAP,ADVANCE AUTO PARTS INC,0.065991,1.000782,0.070460,2,0.999143,11.0,11792.04496,3071.433,0.260467,362.729,Big,Growth


**Q: Why does the table have Stock Exchange Code_x and Stock Exchange Code_y?**

In [46]:
# select useful columns
#final_res = final_res.loc[:, ['Date', 'Ticker', 'Company', 'Annual_Return', 'Group1', 'Group2']]
final_res

,Date,Ticker,Company,S&P_RTN,Equally_wt_RTN,Volume_wt_RTN,S_Period,Annual_Return,Stock Exchange Code,ME,BE,BE/ME,PB,Group1,Group2
0,2015-03-31,AAL,AMERICAN AIRLINES GROUP INC,-0.017396,0.999949,-0.010439,1,1.000188,14.0,36618.81678,2764.000,0.075480,1697.000,Big,Growth
1,2016-03-31,AAL,AMERICAN AIRLINES GROUP INC,0.065991,1.000782,0.070460,2,0.999143,14.0,24017.83458,4710.000,0.196104,1612.000,Big,Growth
2,2015-03-31,AAME,ATLANTIC AMERICAN CORP,-0.017396,0.999949,-0.010439,1,1.000188,14.0,82.93740,110.774,1.335634,1.293,Small,Value
3,2016-03-31,AAME,ATLANTIC AMERICAN CORP,0.065991,1.000782,0.070460,2,0.999143,14.0,90.73995,103.944,1.145515,1.667,Small,Value
4,2015-03-31,AAOI,APPLIED OPTOELECTRONICS INC,-0.017396,0.999949,-0.010439,1,1.000188,14.0,206.71484,114.907,0.555872,0.975,Small,Growth
5,2016-03-31,AAOI,APPLIED OPTOELECTRONICS INC,0.065991,1.000782,0.070460,2,0.999143,14.0,254.51370,165.879,0.651749,1.173,Small,Value
6,2015-03-31,AAON,AAON INC,-0.017396,0.999949,-0.010439,1,1.000188,14.0,1328.81463,195.484,0.147112,16.300,Small,Growth
7,2016-03-31,AAON,AAON INC,0.065991,1.000782,0.070460,2,0.999143,14.0,1485.68000,196.346,0.132159,19.989,Small,Growth
8,2015-03-31,AAP,ADVANCE AUTO PARTS INC,-0.017396,0.999949,-0.010439,1,1.000188,11.0,10952.51792,2146.588,0.195990,356.703,Big,Growth
9,2016-03-31,AAP,ADVANCE AUTO PARTS INC,0.065991,1.000782,0.070460,2,0.999143,11.0,11792.04496,3071.433,0.260467,362.729,Big,Growth


# 5.5 Get Portfolio Returns

In [43]:
portfolio_return = final_res.loc[:, ['Date', 'Group1', 'Group2', 'Annual_Return']].\
groupby(['Date', 'Group1', 'Group2']).mean().reset_index().copy()

In [44]:
portfolio_return['Start_Date'] = portfolio_return['Date'].apply(lambda x: x - pd.DateOffset(years=1))
portfolio_return.columns = ['End_Date', 'S/B', 'V/G', 'Annual_Return', 'Start_Date']
portfolio_return = portfolio_return.loc[:, ['Start_Date', 'End_Date', 'S/B', 'V/G', 'Annual_Return']].copy()
portfolio_return.loc[:, 'Annual_Return'] = (portfolio_return.loc[:, 'Annual_Return'] - 1) * 100

In [45]:
portfolio_return

,Start_Date,End_Date,S/B,V/G,Annual_Return
0,2013-03-31,2014-03-31,Big,Growth,0.238585
1,2013-03-31,2014-03-31,Small,Growth,0.238585
2,2013-03-31,2014-03-31,Small,Value,0.238585
3,2014-03-31,2015-03-31,Big,Growth,0.018757
4,2014-03-31,2015-03-31,Big,Value,0.018757
5,2014-03-31,2015-03-31,Small,Growth,0.018757
6,2014-03-31,2015-03-31,Small,Value,0.018757
7,2015-03-31,2016-03-31,Big,Growth,-0.085701
8,2015-03-31,2016-03-31,Big,Value,-0.085701
9,2015-03-31,2016-03-31,Small,Growth,-0.085701
